# 1. Introduction
We compute the value/rating ratio.

We start by loading the necessary packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# 2. Values

Let us load value data

In [2]:
values = pd.read_pickle('../data/value_records_for_ratings_based_predictions.pkl')
values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9610 entries, 5665 to 160599
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   player_id    9610 non-null   int32         
 1   player_name  9610 non-null   object        
 2   player_role  9607 non-null   category      
 3   birth        9610 non-null   datetime64[ns]
 4   height       9610 non-null   float64       
 5   foot         9610 non-null   object        
 6   value        9610 non-null   float64       
 7   league       9610 non-null   object        
 8   value_at     9610 non-null   datetime64[ns]
 9   nat1         9610 non-null   object        
 10  nat2         9610 non-null   object        
dtypes: category(1), datetime64[ns](2), float64(2), int32(1), object(5)
memory usage: 797.8+ KB


Let us simplify a bit players roles using macro roles

In [3]:
macro_role = {'Goalkeeper':'GK', 
              'Centre-Back':'DF',
              'Defensive Midfield': 'MF',
              'Right Winger':'MF',
              'Centre-Forward':'FW',
              'Right-Back':'DF',
              'Attacking Midfield':'MF',
              'Central Midfield':'MF',
              'Left-Back':'DF',
              'Left Winger':'MF',
              'Right Midfield':'MF',
              'Left Midfield':'MF',
              'Second Striker':'FW',
              'Attacking Midfield':'MF'
             }
values["macro_role"] = values["player_role"].apply(lambda x: macro_role.get(x))
values.head()

,player_id,player_name,player_role,birth,height,foot,value,league,value_at,nat1,nat2,macro_role
5665,94308,Marko Dmitrovic,Goalkeeper,1992-01-24,1.94,left,3.60,SPA1,2018-06-30,Serbia,-,GK
5667,139336,Paulo Oliveira,Centre-Back,1992-01-08,1.87,right,2.70,SPA1,2018-06-30,Portugal,-,DF
5671,87469,JosÃ© Ãngel,Left-Back,1989-09-05,1.82,left,2.25,SPA1,2018-06-30,Spain,-,DF
5674,266795,Gonzalo Escalante,Central Midfield,1993-03-27,1.82,right,2.25,SPA1,2018-06-30,Argentina,Italy,MF
5676,153427,BebÃ©,Left Winger,1990-07-12,1.90,right,0.90,SPA1,2018-06-30,Portugal,CapeVerde,MF


For a number of players in the values records there are no ratings statistics. This might be due to the fact that they have not made an appearence in official matches. We filter them out.

In [4]:
missing = pd.read_csv('../data/missing.csv')
len(missing)

41

In [5]:
len(values)

9610

In [6]:
values = values.loc[~values['player_id'].isin(missing['ID']), :]
len(values) 

9556

This number is smaller than 9610 - 41 = 9559 as some players may be in more records, i.e., have value for different years.

In [7]:
values['year'] = values['value_at'].dt.year
values

,player_id,player_name,player_role,birth,height,foot,value,league,value_at,nat1,nat2,macro_role,year
5665,94308,Marko Dmitrovic,Goalkeeper,1992-01-24,1.94,left,3.60,SPA1,2018-06-30,Serbia,-,GK,2018
5667,139336,Paulo Oliveira,Centre-Back,1992-01-08,1.87,right,2.70,SPA1,2018-06-30,Portugal,-,DF,2018
5671,87469,JosÃ© Ãngel,Left-Back,1989-09-05,1.82,left,2.25,SPA1,2018-06-30,Spain,-,DF,2018
5674,266795,Gonzalo Escalante,Central Midfield,1993-03-27,1.82,right,2.25,SPA1,2018-06-30,Argentina,Italy,MF,2018
5676,153427,BebÃ©,Left Winger,1990-07-12,1.90,right,0.90,SPA1,2018-06-30,Portugal,CapeVerde,MF,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...
160590,138927,Daniel Carvajal,Right-Back,1992-01-11,1.73,right,10.80,SPA1,2014-06-30,Spain,-,DF,2014
160594,31909,Toni Kroos,Central Midfield,1990-01-04,1.83,both,36.00,SPA1,2014-06-30,Germany,-,MF,2014
160597,88103,James RodrÃ­guez,Attacking Midfield,1991-07-12,1.81,left,31.50,SPA1,2014-06-30,Colombia,Spain,MF,2014
160598,8198,Cristiano Ronaldo,Left Winger,1985-02-05,1.87,both,90.00,SPA1,2014-06-30,Portugal,-,MF,2014


# 3. Ratings
Let us now load the ratings values for different ages.

In [8]:
dateparser = lambda x: datetime.strptime(x, '%d.%m.%Y')
ratings_17 = pd.read_csv('../data/ratings_v2_17.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)
ratings_18 = pd.read_csv('../data/ratings_v2_18.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)
ratings_19 = pd.read_csv('../data/ratings_v2_19.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)
ratings_20 = pd.read_csv('../data/ratings_v2_20.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)
ratings_21 = pd.read_csv('../data/ratings_v2_21.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)
ratings_22 = pd.read_csv('../data/ratings_v2_22.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)
ratings_23 = pd.read_csv('../data/ratings_v2_23.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)
ratings_24 = pd.read_csv('../data/ratings_v2_24.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)
ratings_25 = pd.read_csv('../data/ratings_v2_25.txt',names = ['player_id','player_name','birth_date','rating','peak_rating','minutes_played','value_at'], index_col=False,parse_dates=['birth_date','value_at'],date_parser=dateparser)

In [9]:
ratings_17.head()

,player_id,player_name,birth_date,rating,peak_rating,minutes_played,value_at
0,94308,Marko Dmitrovic,1992-01-24,NaN,NaN,0,1900-01-01
1,139336,Paulo Oliveira,1992-01-08,NaN,NaN,0,1900-01-01
2,87469,Jose Angel,1989-09-05,NaN,NaN,0,1900-01-01
3,266795,Gonzalo Escalante,1993-03-27,NaN,NaN,0,1900-01-01
4,153427,Bebe,1990-07-12,NaN,NaN,0,1900-01-01


From these records let us filter out missing values.

In [10]:
ratings_17 = ratings_17.dropna(subset=['rating', 'peak_rating'])
ratings_18 = ratings_18.dropna(subset=['rating', 'peak_rating'])
ratings_19 = ratings_19.dropna(subset=['rating', 'peak_rating'])
ratings_20 = ratings_20.dropna(subset=['rating', 'peak_rating'])
ratings_21 = ratings_21.dropna(subset=['rating', 'peak_rating'])
ratings_22 = ratings_22.dropna(subset=['rating', 'peak_rating'])
ratings_23 = ratings_23.dropna(subset=['rating', 'peak_rating'])
ratings_24 = ratings_24.dropna(subset=['rating', 'peak_rating'])
ratings_25 = ratings_25.dropna(subset=['rating', 'peak_rating'])

In [11]:
ratings = pd.concat([ratings_17,ratings_18,ratings_19,ratings_20,ratings_21,ratings_22,ratings_23,ratings_24,ratings_25])
ratings['year'] = ratings['value_at'].dt.year
ratings

,player_id,player_name,birth_date,rating,peak_rating,minutes_played,value_at,year
50,26199,Verza,1986-09-29,0.034665,0.141346,1,2003-08-31,2003
58,221321,Joaquin Fernandez,1996-05-31,-0.191357,0.008579,56,2013-04-30,2013
71,251106,Enes Ünal,1997-05-10,-0.142142,0.063741,459,2014-04-30,2014
109,199321,Ruben Blanco,1995-07-25,-0.218329,-0.012563,360,2012-06-30,2012
110,297194,Ivan Villar,1997-07-09,-0.206440,0.005668,91,2014-06-30,2014
...,...,...,...,...,...,...,...,...
3557,39381,Gareth Frank Bale,1989-07-16,0.211451,0.223058,20519,2014-06-30,2014
3558,316264,Martin Odegaard,1998-12-17,0.005114,0.029365,18284,2021-11-08,2021
3559,18922,Karim Benzema,1987-12-19,0.264356,0.270048,16420,2012-11-30,2012
3560,298976,Borja Mayoral Moya,1997-04-05,0.025522,0.034916,10612,2021-11-08,2021


# 4. Value/Rating ratio
Let us now calculate the value/rating ratio. Instead of using market values and ratings, which are dimensionally different, we can use their ranking, that is value_rank/rating_rank. However, this calculation introduces new challenges: how do we rank players? For example, given a moment in time, we could:
- rank all active players at that time
- rank all active players at that time by age group (this would be more fair since age has an impact at least on market values)
- rank all active players at that time by age group and role group (this would be more fair since age and role have an impact at least on market values)

We start by calculating V/R ratio without any particular classification of players. That is, we take all the active players at a given season date and rank them according to value and rating.

Let us create a function that does all these steps and returns the $n$ best players in terms of V/R ratio

In [20]:
def get_best_players(values:pd.DataFrame, ratings:pd.DataFrame,year:int,ranked_by='rating'):
    # First it selects the records for the appropriate year
    v = values.loc[values['year']  == year,['player_id','player_name','macro_role','birth','value']]
    r = ratings.loc[ratings['year']  == year,['player_id','rating','peak_rating','minutes_played']]
    data = r.merge(v,on='player_id', how='inner')
    data['rank_'+ranked_by] = data[ranked_by].rank(pct=True,ascending=False,method='average',na_option='bottom')
    data['rank_value'] = data['value'].rank(pct=True,ascending=False,method='average',na_option='bottom')
    data['V/R_'+ranked_by] = data.apply(lambda row: row['rank_value']/row['rank_'+ranked_by],axis=1)
    data = data.sort_values(by=['V/R_'+ranked_by],ascending=False)
    return data

In [21]:
p14 = get_best_players(values = values,ratings=ratings,year=2014,ranked_by='peak_rating')
p14

,player_id,rating,peak_rating,minutes_played,player_name,macro_role,birth,value,rank_peak_rating,rank_value,V/R_peak_rating
248,138927,0.254860,0.280629,13619,Daniel Carvajal,DF,1992-01-11,10.800,0.008152,0.156250,19.166667
173,145922,0.182354,0.205319,3244,Jon Flanagan,DF,1993-01-01,0.900,0.040761,0.724185,17.766667
8,243714,0.015559,0.187348,40,Kingsley Coman,MF,1996-06-13,0.675,0.062500,0.792120,12.673913
26,202886,0.034549,0.162664,10,Ruben Loftus-Cheek,MF,1996-01-23,0.135,0.108696,0.964674,8.875000
175,80444,0.190761,0.227242,6685,Philippe Coutinho,MF,1992-06-12,14.400,0.019022,0.127717,6.714286
...,...,...,...,...,...,...,...,...,...,...,...
356,30059,0.086545,0.098798,12320,Lars Bender,MF,1989-04-27,19.800,0.326087,0.059783,0.183333
165,111630,0.044126,0.083286,8059,Erik Lamela,MF,1992-03-04,21.600,0.426630,0.052989,0.124204
106,122153,0.112606,0.172618,5609,Paul Pogba,MF,1993-03-15,40.500,0.089674,0.010870,0.121212
367,39381,0.211451,0.223058,20519,Gareth Bale,MF,1989-07-16,72.000,0.024457,0.002717,0.111111


Ideally we want a *high value rank*, meaning a low value, and a *low rating rank*, meaning a high rating. Thus, we look for players with *high V/R ratios*. 
Let us get the top and the bottom playees, and assess how their values progress after one or two seasons.

In [22]:
p14top = p14.head(n=10)
p14bottom = p14.tail(n=10)

Let us see the value change after two seasons.

In [23]:
change_top = p14top.loc[:,['player_id','player_name','birth','value','rating']].merge(values.loc[values['year']==2016,['player_id','player_name','birth','value']],on='player_id', how='inner')
change_top['value_change'] = change_top.apply(lambda row:100* (row['value_y']-row['value_x'])/row['value_x'],axis=1)
change_top

,player_id,player_name_x,birth_x,value_x,rating,player_name_y,birth_y,value_y,value_change
0,138927,Daniel Carvajal,1992-01-11,10.800,0.254860,Daniel Carvajal,1992-01-11,22.50,108.333333
1,145922,Jon Flanagan,1993-01-01,0.900,0.182354,Jon Flanagan,1993-01-01,6.30,600.000000
2,202886,Ruben Loftus-Cheek,1996-01-23,0.135,0.034549,Ruben Loftus-Cheek,1996-01-23,4.50,3233.333333
3,80444,Philippe Coutinho,1992-06-12,14.400,0.190761,Philippe Coutinho,1992-06-12,28.80,100.000000
4,177476,Nathan AkÃ©,1995-02-18,1.350,0.020238,Nathan AkÃ©,1995-02-18,4.05,200.000000
5,241481,Jairo Riedewald,1996-09-09,0.225,-0.045992,Jairo Riedewald,1996-09-09,4.95,2100.000000
6,57500,CÃ©sar Azpilicueta,1989-08-28,16.200,0.217135,CÃ©sar Azpilicueta,1989-08-28,18.00,11.111111
7,182913,Adrien Rabiot,1995-04-03,3.150,0.035125,Adrien Rabiot,1995-04-03,9.00,185.714286
8,128912,Andre Wisdom,1993-05-09,2.250,0.089297,Andre Wisdom,1993-05-09,3.60,60.000000


In [24]:
change_bottom = p14bottom.loc[:,['player_id','player_name','birth','value','rating']].merge(values.loc[values['year']==2017,['player_id','player_name','birth','value']],on='player_id', how='inner')
change_bottom['value_change'] = change_bottom.apply(lambda row:100* (row['value_y']-row['value_x'])/row['value_x'],axis=1)
change_bottom

,player_id,player_name_x,birth_x,value_x,rating,player_name_y,birth_y,value_y,value_change
0,46001,Moussa Sissoko,1989-08-16,9.9,0.053408,Moussa Sissoko,1989-08-16,18.0,81.818182
1,88755,Kevin De Bruyne,1991-06-28,15.3,0.053285,Kevin De Bruyne,1991-06-28,58.5,282.352941
2,90681,Eliaquim Mangala,1991-02-13,20.7,0.090315,Eliaquim Mangala,1991-02-13,13.5,-34.782609
3,110923,Mattia Perin,1992-11-10,9.0,-0.037957,Mattia Perin,1992-11-10,13.5,50.000000
4,30059,Lars Bender,1989-04-27,19.8,0.086545,Lars Bender,1989-04-27,9.0,-54.545455
5,111630,Erik Lamela,1992-03-04,21.6,0.044126,Erik Lamela,1992-03-04,22.5,4.166667
6,122153,Paul Pogba,1993-03-15,40.5,0.112606,Paul Pogba,1993-03-15,67.5,66.666667
7,39381,Gareth Bale,1989-07-16,72.0,0.211451,Gareth Bale,1989-07-16,72.0,0.000000
8,68290,Neymar,1992-02-05,54.0,0.138501,Neymar,1992-02-05,90.0,66.666667


It seems that the players with a higher V/R ratio have a better chance to see their values increasing.

-----------------------------
## Now we try to group players by age.

In [68]:
age = 20
year = 2014
birth = year - age

In [69]:
r = ratings[age]

In [70]:
v = values.loc[(values['birth']> str(birth)+'-1-1') 
                        & (values['birth'] < str(birth)+'-12-31') 
                        & ( (values['value_at'].dt.year - values['birth'].dt.year) == age),:]
#v = v.loc[v['macro_role'] == "FW",['player_id','player_name','player_role','macro_role','birth','value']]
v = v.loc[:,['player_id','player_name','player_role','macro_role','birth','value']]

In [71]:
data = r.merge(v,on='player_id', how='inner')

In [72]:
data['rank_rating'] = data['rating'].rank(pct=True,ascending=False,method='average',na_option='bottom')

In [73]:
data['rank_value'] = data['value'].rank(pct=True,ascending=False,method='average',na_option='bottom')

In [74]:
data['V/R'] = data.apply(lambda row: row['rank_value']/row['rank_rating'],axis=1)